In [15]:
import numpy as np
import pandas as pd
import pickle

# Install folium if needed:
#!pip install folium
import folium

In [4]:
with open('turnstiles_full.pickle', 'rb') as file:
    mta_full = pickle.load(file)
with open('turnstiles_by_day.pickle', 'rb') as file2:
    by_day = pickle.load(file2)
by_day.head()

,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATE,PREV_ENTRIES,PREV_EXITS,weekday,daily_entries,daily_exits
1,A002,R051,02-00-00,59 ST,7027457,2382455,2019-04-20,7026702.0,2382234.0,Saturday,755.0,221.0
2,A002,R051,02-00-00,59 ST,7028053,2382642,2019-04-21,7027457.0,2382455.0,Sunday,596.0,187.0
3,A002,R051,02-00-00,59 ST,7029313,2383025,2019-04-22,7028053.0,2382642.0,Monday,1260.0,383.0
4,A002,R051,02-00-00,59 ST,7030821,2383462,2019-04-23,7029313.0,2383025.0,Tuesday,1508.0,437.0
5,A002,R051,02-00-00,59 ST,7032351,2383896,2019-04-24,7030821.0,2383462.0,Wednesday,1530.0,434.0


In [6]:
by_day['total_traffic'] = by_day['daily_entries'] + by_day['daily_exits']
station_total = by_day.groupby('STATION')['total_traffic']\
    .sum().reset_index().sort_values('total_traffic', ascending=False)
station_total.head(10)

,STATION,total_traffic
61,34 ST-PENN STA,15308823.0
232,GRD CNTRL-42 ST,12880149.0
59,34 ST-HERALD SQ,10874560.0
14,14 ST-UNION SQ,9625476.0
352,TIMES SQ-42 ST,9468584.0
46,23 ST,9135850.0
225,FULTON ST,8360167.0
68,42 ST-PORT AUTH,8323960.0
110,86 ST,7606546.0
9,125 ST,6712682.0


In [7]:
loc_url = 'http://web.mta.info/developers/data/nyct/subway/Stations.csv'
mta_loc = pd.read_csv(loc_url)
mta_loc.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan


In [8]:
mta_loc.rename(columns={'Stop Name': 'STATION', 'GTFS Latitude': 'LAT', 'GTFS Longitude': 'LONG'}, inplace = True)
mta_loc['STATION'] = mta_loc.STATION.str.upper()
mta_loc['STATION'] = mta_loc.STATION.str.replace(' - ', '-')
mta_loc = mta_loc.drop_duplicates('STATION').drop(columns = ['Station ID', 'Complex ID', 'GTFS Stop ID',
                                                            'Division', 'Structure', 'North Direction Label',
                                                            'South Direction Label'])

mta_loc.head()

,Line,STATION,Borough,Daytime Routes,LAT,LONG
0,Astoria,ASTORIA-DITMARS BLVD,Q,N W,40.775036,-73.912034
1,Astoria,ASTORIA BLVD,Q,N W,40.770258,-73.917843
2,Astoria,30 AV,Q,N W,40.766779,-73.921479
3,Astoria,BROADWAY,Q,N W,40.761820,-73.925508
4,Astoria,36 AV,Q,N W,40.756804,-73.929575


In [9]:
mta_loc.STATION = mta_loc.STATION.replace('34 ST-PENN STATION', '34 ST-PENN STA')
mta_loc.STATION = mta_loc.STATION.replace('GRAND CENTRAL-42 ST', 'GRD CNTRL-42 ST')
mta_loc.STATION = mta_loc.STATION.replace('42 ST-PORT AUTHORITY BUS TERMINAL', '42 ST-PORT AUTH')
mta_loc.STATION = mta_loc.STATION.replace('59 ST-COLUMBUS CIRCLE', '59 ST COLUMBUS')
mta_loc.STATION = mta_loc.STATION.replace('47-50 STS-ROCKEFELLER CTR', '47-50 STS ROCK')
mta_loc.STATION = mta_loc.STATION.replace('FLUSHING-MAIN ST', 'FLUSHING-MAIN')
mta_loc.STATION = mta_loc.STATION.replace('JACKSON HTS-ROOSEVELT AV', 'JKSN HT-ROOSVLT')

In [19]:
mta_join = by_day.merge(mta_loc, how = 'inner', on = 'STATION')
print(mta_join.shape, by_day.shape)
mta_join.head()

(180057, 18) (262213, 13)


,C/A,UNIT,SCP,STATION,ENTRIES,EXITS,DATE,PREV_ENTRIES,PREV_EXITS,weekday,daily_entries,daily_exits,total_traffic,Line,Borough,Daytime Routes,LAT,LONG
0,A002,R051,02-00-00,59 ST,7027457,2382455,2019-04-20,7026702.0,2382234.0,Saturday,755.0,221.0,976.0,4th Av,Bk,N R,40.641362,-74.017881
1,A002,R051,02-00-00,59 ST,7028053,2382642,2019-04-21,7027457.0,2382455.0,Sunday,596.0,187.0,783.0,4th Av,Bk,N R,40.641362,-74.017881
2,A002,R051,02-00-00,59 ST,7029313,2383025,2019-04-22,7028053.0,2382642.0,Monday,1260.0,383.0,1643.0,4th Av,Bk,N R,40.641362,-74.017881
3,A002,R051,02-00-00,59 ST,7030821,2383462,2019-04-23,7029313.0,2383025.0,Tuesday,1508.0,437.0,1945.0,4th Av,Bk,N R,40.641362,-74.017881
4,A002,R051,02-00-00,59 ST,7032351,2383896,2019-04-24,7030821.0,2383462.0,Wednesday,1530.0,434.0,1964.0,4th Av,Bk,N R,40.641362,-74.017881


In [40]:
m = folium.Map([40.75, -73.91], zoom_start=12)
for index, row in mta_join[mta_join.DATE=='2019-06-13'].iterrows():
    folium.CircleMarker([row['LAT'], row['LONG']], radius = 8, color = 'black').add_to(m)
for index, row in mta_join[(mta_join.DATE=='2019-06-13') & 
                           (mta_join.STATION.isin(station_total.STATION[0:5]))].iterrows():
    folium.CircleMarker([row['LAT'], row['LONG']], radius = 8, color = 'red').add_to(m)
for index, row in mta_join[(mta_join.DATE=='2019-06-13') & 
                           (mta_join.STATION.isin(station_total.STATION[5:10]))].iterrows():
    folium.CircleMarker([row['LAT'], row['LONG']], radius = 8, color = 'orange').add_to(m)
for index, row in mta_join[(mta_join.DATE=='2019-06-13') & 
                           (mta_join.STATION.isin(station_total.STATION[10:15]))].iterrows():
    folium.CircleMarker([row['LAT'], row['LONG']], radius = 8, color = 'yellow').add_to(m)
for index, row in mta_join[(mta_join.DATE=='2019-06-13') & 
                           (mta_join.STATION.isin(station_total.STATION[15:20]))].iterrows():
    folium.CircleMarker([row['LAT'], row['LONG']], radius = 8, color = 'green').add_to(m)
for index, row in mta_join[(mta_join.DATE=='2019-06-13') & 
                           (mta_join.STATION.isin(station_total.STATION[25:30]))].iterrows():
    folium.CircleMarker([row['LAT'], row['LONG']], radius = 8, color = 'blue').add_to(m)

m.save('station_map.html')
m